In [1]:
import nibabel as nib
import numpy as np
from tqdm.auto import tqdm

/home/student/sinaziaee/mini_conda/miniconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

def make_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path
new_folder_path = '3d_zero_padded_images'
dest_train_images_path = make_path(f'{new_folder_path}/training/images')
dest_train_masks_path = make_path(f'{new_folder_path}/training/labels')
dest_valid_images_path = make_path(f'{new_folder_path}/validation/images')
dest_valid_masks_path = make_path(f'{new_folder_path}/validation/labels')
dest_test_images_path = make_path(f'{new_folder_path}/testing/images')
dest_test_masks_path = make_path(f'{new_folder_path}/testing/labels')

src_train_images_path = '/scratch/student/sinaziaee/datasets/3d_dataset/training/images'
src_train_masks_path = '/scratch/student/sinaziaee/datasets/3d_dataset/training/labels'
src_valid_images_path = '/scratch/student/sinaziaee/datasets/3d_dataset/validation/images'
src_valid_masks_path = '/scratch/student/sinaziaee/datasets/3d_dataset/validation/labels'
src_test_images_path = '/scratch/student/sinaziaee/datasets/3d_dataset/testing/images'
src_test_masks_path = '/scratch/student/sinaziaee/datasets/3d_dataset/testing/labels'

In [5]:
def pad_image(image, mask, dest_image_path, dest_mask_path, new_image_name, new_mask_name):
    desired_depth = 512
    padding_needed = desired_depth - image.shape[2]
    pad_before = padding_needed // 2
    pad_after = padding_needed - pad_before
    padded_img = np.pad(image, ((0, 0), (0, 0), (pad_before, pad_after)), mode='constant', constant_values=0)
    padded_mask = np.pad(mask, ((0, 0), (0, 0), (pad_before, pad_after)), mode='constant', constant_values=0)
    new_image_nifti = nib.Nifti1Image(padded_img, np.eye(4))
    new_mask_nifti = nib.Nifti1Image(padded_mask, np.eye(4))
    nib.save(new_image_nifti, f'{dest_image_path}/{new_image_name}')
    nib.save(new_mask_nifti, f'{dest_mask_path}/{new_mask_name}')

In [6]:
def padding_3d_image(src_image_path, src_mask_path, dest_image_path, dest_mask_path):
    for image_file_name, mask_file_name in zip(sorted(os.listdir(src_image_path)), sorted(os.listdir(src_mask_path))):
        image = nib.load(os.path.join(src_image_path, image_file_name)).get_fdata()
        mask = nib.load(os.path.join(src_mask_path, mask_file_name)).get_fdata()
        pad_image(image, mask, dest_image_path, dest_mask_path, image_file_name, mask_file_name)
padding_3d_image(src_train_images_path, src_train_masks_path, dest_train_images_path, dest_train_masks_path)
padding_3d_image(src_valid_images_path, src_valid_masks_path, dest_valid_images_path, dest_valid_masks_path)
padding_3d_image(src_test_images_path, src_test_masks_path, dest_test_images_path, dest_test_masks_path)